<a href="https://colab.research.google.com/github/remi-boivin/own-gpt/blob/google_colab/own-gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


wocabulary

In [13]:
with open("wizard_of_oz.txt", encoding="utf-8") as f:
  text = f.read()
chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

[' ', '!', '"', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '·', '\ufeff']


encoder

In [14]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([83,  2, 72, 57, 81, 68, 71, 57, 60,  2, 23, 83,  2, 57, 68, 68, 45, 64,
        71, 74, 76, 59, 77, 76, 75, 31, 70, 57, 58, 68, 61, 60,  2, 23, 62, 57,
        68, 75, 61,  9,  2, 62, 65, 68, 61, 46, 74, 61, 61,  2, 23, 83,  2,  2,
        23, 83,  2, 65, 76, 61, 69, 75,  2, 23, 53, 83,  2, 70, 57, 69, 61,  2,
        23,  2, 44, 31, 27, 30, 39, 31, 11, 69, 60,  2,  9,  2, 72, 57, 76, 64,
         2, 23,  2, 44, 31, 27, 30, 39, 31, 11])


In [17]:
n = int(0.8 * len(data))
train_data = data[:n]
valid_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size, ))
  print(ix)
  x = torch.stack([data[i:i + block_size] for i in ix])
  y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

x,y = get_batch('train')
print(f"inputs: {x}\n targets {y}")

tensor([118236, 151725,  72622, 193904])
inputs: tensor([[77, 75, 61, 75,  0, 65, 70,  0],
        [ 0, 33, 77, 74, 63, 68, 61, 75],
        [64, 65, 75,  0, 65, 70, 75, 65],
        [71, 81,  0, 75, 76, 71, 72, 72]], device='cuda:0')
 targets tensor([[75, 61, 75,  0, 65, 70,  0, 76],
        [33, 77, 74, 63, 68, 61, 75,  0],
        [65, 75,  0, 65, 70, 75, 65, 60],
        [81,  0, 75, 76, 71, 72, 72, 61]], device='cuda:0')


In [3]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, target):
    logits = self.token_embedding_table(index)
    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(B * T)
    loss = F.cross_entropy(logits, target)
    return logits


In [10]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
  context = x[:t + 1]
  target = y[t]
  print('when input is', context, 'target is', target)

when input is tensor([83]) target is tensor(2)
when input is tensor([83,  2]) target is tensor(72)
when input is tensor([83,  2, 72]) target is tensor(57)
when input is tensor([83,  2, 72, 57]) target is tensor(81)
when input is tensor([83,  2, 72, 57, 81]) target is tensor(68)
when input is tensor([83,  2, 72, 57, 81, 68]) target is tensor(71)
when input is tensor([83,  2, 72, 57, 81, 68, 71]) target is tensor(57)
when input is tensor([83,  2, 72, 57, 81, 68, 71, 57]) target is tensor(60)
